<font color='tomato'><font color="#CC3D3D"><p>
# A DNN Model for Multiclass Classification

##### Import modules

In [1]:
import pandas as pd
import numpy as np
import os
import random
import pickle
from tqdm import tqdm
from IPython.display import Image, clear_output
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
%matplotlib inline

from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import kerastuner as kt
print(tf.__version__)

2.8.0


/tmp/ipykernel_21949/3188710588.py:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


##### Set random seeds to make your results reproducible

In [2]:
# 매번 모델링을 할 때마다 동일한 결과를 얻으려면 아래 코드를 실행해야 함.

def reset_seeds(s1,s2,s3, reset_graph_with_backend=None):
    if reset_graph_with_backend is not None:
        K = reset_graph_with_backend
        K.clear_session()
        tf.compat.v1.reset_default_graph()
        print("KERAS AND TENSORFLOW GRAPHS RESET")  # optional

    np.random.seed(s1)
    random.seed(s2)
    tf.compat.v1.set_random_seed(s3)
#    os.environ['CUDA_VISIBLE_DEVICES'] = ''  # for GPU
#    print("RANDOM SEEDS RESET")  # optional

### Step 1: Load and process the data

##### Read data

In [3]:
df_train = pd.read_csv('X_train.csv', encoding='cp949')
df_test = pd.read_csv('X_test.csv', encoding='cp949')
y_train = pd.read_csv('y_train.csv').group
IDtest = df_test.custid.unique()

df_train.head()

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,0,6,25,일,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,90000,9000,81000,3,0
1,0,6,25,일,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,39000,3900,35100,1,0
2,0,8,26,토,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,1,175000,17500,157500,3,0
3,0,8,26,토,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,잡화파트,잡화가용팀,수입명품,1,455000,45500,409500,3,0
4,0,9,3,일,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,0,100000,10000,90000,3,0


##### Transform Data with Item Frequency

In [4]:
level = 'corner_nm'
df_train[level].nunique()

308

In [5]:
df_all = pd.concat([df_train, df_test])
X_train = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc=np.size, fill_value=0). \
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid']).values
X_test = pd.pivot_table(df_all, index='custid', columns=level, values='tot_amt',
                         aggfunc=np.size, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid']).values

##### One-hot-encode Target variable 

In [6]:
# 8개의 범주형 타겟 값을 one-hot-encoding을 통해 8개의 컬럼으로 만들어야 함.
y_train = keras.utils.to_categorical(y_train.astype('category').cat.codes)

##### Split data into train & validation set 

In [7]:
i = int(round(X_train.shape[0] * 0.8,0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

##### Feature scaling

In [8]:
# DNN은 scale된 값을 입력값으로 사용해야 함.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

### Step 2: Define the hyper-model

In [9]:
def model_fn(hp):
    inputs = keras.Input(shape=(X_train.shape[1],))
    x = inputs
    for i in range(hp.Int('num_layers', 2, 3)):
        x = keras.layers.Dense(hp.Int('unit_'+str(i), 16, 64, step=16), activation='relu')(x)
        x = keras.layers.Dropout(hp.Float('dropout_'+str(i), 0, 0.5, step=0.25, default=0.5))(x)
    outputs = keras.layers.Dense(8, activation='softmax')(x) # 예측값이 8종류이므로 8개 출력 뉴런 필요
    model = keras.Model(inputs, outputs)
    model.compile(loss='categorical_crossentropy', # Multiclass Classification에서 사용하는 loss function
                  optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), 
                  metrics=[keras.metrics.CategoricalCrossentropy()]) # Multiclass Classification에서 사용하는 평가지표
    return model

### Step 3: Build multiple hyper-tuned models

In [10]:
tuner = kt.Hyperband(model_fn,
                     objective=kt.Objective('val_categorical_crossentropy', direction="min"), 
                     max_epochs=10,
                     hyperband_iterations=2,
                     overwrite=True,
                     directory='dnn_tuning')
tuner.search(X_train, y_train, validation_data=(X_valid, y_valid), 
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])
model = tuner.get_best_models(1)[0]  
tuner.results_summary(1)

Trial 60 Complete [00h 00m 07s]
val_categorical_crossentropy: 1.775173544883728

Best val_categorical_crossentropy So Far: 1.642884612083435
Total elapsed time: 00h 13m 42s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in dnn_tuning/untitled_project
Showing 1 best trials
Objective(name='val_categorical_crossentropy', direction='min')
Trial summary
Hyperparameters:
num_layers: 3
unit_0: 48
dropout_0: 0.25
unit_1: 48
dropout_1: 0.25
learning_rate: 0.001
unit_2: 48
dropout_2: 0.25
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 6abada107017dc352f974d9f09f85fa9
Score: 1.642884612083435


### Step 4: Make submissions

In [11]:
t = pd.Timestamp.now()
fname = f"dnn_submission_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
pred = pd.DataFrame(model.predict(X_test))
pred.columns = ['F20','F30','F40','F50','M20','M30','M40','M50']
pd.concat([pd.Series(IDtest, name="ID"), pred] ,axis=1).to_csv(fname, index=False)
print(f"'{fname}' is ready to submit.")

'dnn_submission_06071334.csv' is ready to submit.


<font color="#CC3D3D"><p>
# End